In [1]:
from pprint import pprint
import json
from src.URESTAdapter import URESTAdapter
from src.utils.OASParser import OASParser


Инициализация Продюссера

In [ ]:
Определяем сущность (пока временно это operationID),он же будет топиком в кафку

In [2]:
TOPIC_NAME = 'get_everything'
    #  forecast - Яндекс Погода
    #  getTopHeadlines - API новостной с пагинацей
    #  getEverything - API новостной с пагинацей
    #  suggestions_dadata_suggestBank - Dadata - сервис API

src_dict = {'get_everything':'C:/Users/kdenis/Documents/Work/OpenApiSpecParser/examples/NewsAPI.yaml'
                ,'post_api_4_1_rs_suggest_bank':'C:/Users/kdenis/Documents/Work/OpenApiSpecParser/examples/suggestions.yml'
                ,'get_top-headlines':'C:/Users/kdenis/Documents/Work/OpenApiSpecParser/examples/NewsAPI.yaml'
                ,'get_currentconditions_v1__locationKey_':'C:/Users/kdenis/Documents/Work/OpenApiSpecParser/examples/accuweather.yaml'}
    
input_data = {
    'get_everything':{'q':'RUSSIA'},
    'post_api_4_1_rs_suggest_bank':[
        'SABRRUMM', 'VTBRRUMM', 'GAZPRUMM', 'ALFARUMM', 'MOSCRUMM',
        'RSBNRUMM', 'RUWCRUMM', 'ICBKRUMM', 'KOSKRUMM', 'PARNRUMM',
        'ABNYRUMM', 'CRYPRUMM', 'TICSRUMM', 'PSBZRUMM', 'TKRBRUMM',
        'JSNMRUMM', 'MIRBRUMM', 'ELSRUMMXXX', 'RNGBRUMM', 'IRONRUMM',
        'AVJSRUMM', 'ARESRUMM', 'ALILRUMM', 'ITRORU8Y', 'BOCSRUMM',
        'DOMRRUMM', 'FORTRUMM', 'GLBKRUMM', 'HDCBRUMM', 'KBKTRUMM',
        'KREMRUMM', 'LBRURUMM', 'MDMBRUMM', 'MEZHRUMM', 'MOPARUMM',
        'OLMDRUMM', 'ROYCRUMM', 'RZCBRUMM', 'SBERRUMM', 'SGBZRUMM',
        'SLAVRUMM', 'SOGZRUMM', 'TATKRUMM', 'TKBKRUMM', 'TKZLRUMM',
        'TKZVRUMM', 'TRNVRUMM', 'VEFKRUMM', 'VTBKRUMM', 'ZENIRUMM'
    ],
    'get_top-headlines':{'q':'RUSSIA'},
    'get_currentconditions_v1__locationKey_':[1,2,3,45,66666,77777,9999,88]
}

In [3]:
parser = OASParser(src_dict[TOPIC_NAME])
entity  = parser.request
pprint(entity)

{'get_everything': {'auth_types': [{'apiKey': []}],
                    'base_url': 'https://newsapi.org/v2',
                    'headers': {'Accept': 'application/json',
                                'Content-Type': 'application/json'},
                    'method': 'GET',
                    'operationalId': 'get_everything',
                    'required': [],
                    'url': 'https://newsapi.org/v2/everything',
                    'variables': ['domains',
                                  'excludeDomains',
                                  'from',
                                  'language',
                                  'page',
                                  'pageSize',
                                  'q',
                                  'searchIn',
                                  'sortBy',
                                  'sources',
                                  'to']},
 'get_sources': {'auth_types': [{'apiKey': []}],
                 'base_url': 

Swift-банков

Инициализция Парсера + Адаптера - различные варианты:

In [8]:
parser = OASParser(src_dict[TOPIC_NAME])
entity  = parser.request.get(TOPIC_NAME)
'''Mock сервера ключей'''
with open('C:/Users/kdenis/mu_code/keys.json', 'r', encoding='utf-8') as f:
    tokens = json.load(f)
token = tokens.get(entity.get('base_url'))
      

test = URESTAdapter(entity,token)
result = test.execute(input_data.get(TOPIC_NAME))
pprint(result)

переменная(ые) есть в списке
[{'articles': [{'author': 'Matthew Gault',
                'content': 'For half a century, the worlds nuclear powers '
                           'relied on an intricate and complex series of '
                           'treaties that slowly and steadily reduced the '
                           'number of nuclear weapons on the planet. Those '
                           'treaties are go… [+3542 chars]',
                'description': 'The last major nuclear arms treaty between the '
                               'US and Russia just expired. Some experts '
                               'believe a combination of satellite '
                               'surveillance, AI, and human reviewers can take '
                               'its place. Others, not so much.',
                'publishedAt': '2026-02-09T11:30:00Z',
                'source': {'id': 'wired', 'name': 'Wired'},
                'title': 'AI Is Here to Replace Nuclear Treaties. Scared Yet